In [ ]:
import csv as csv
import numpy as np

In [ ]:
csv_file = csv.reader(open('train.csv', 'rb'))
header = csv_file.next()
print header

In [ ]:
data = []
for row in csv_file:
    data.append(row)
npdata = np.array(data) #convet list to an array

In [ ]:
print npdata[0]  #exploring fields
print npdata[::, 9]

In [ ]:
number_passengers = np.size(npdata[::,1].astype(np.float))
number_survived = np.sum(npdata[::,1].astype(np.float))
proportion_survived = number_survived / number_passengers
print 'number of passengers:', number_passengers, '\n', 'number of survived:', number_survived, \
'\n', 'proportion survived:', proportion_survived 


In [ ]:
women_stats = npdata[::,4] == 'female'
men_stats = npdata[::,4] =='male'

In [ ]:
women_onboard = npdata[women_stats, 1].astype(np.float)
men_onboard = npdata[men_stats, 1].astype(np.float)

In [ ]:
women_survived = np.sum(women_onboard)/np.size(women_stats)
men_survived = np.sum(men_onboard)/np.size(men_stats)
print 'Proportion of women who survived is', women_survived
print 'Proportion of men who survived is', men_survived

In [ ]:
test_file = open('train.csv', 'rb')
test_file_object = csv.reader(test_file)
header = test_file_object.next()

In [ ]:
prediction_file = open('gendermodel.csv', 'wb')
prediction_file_object = csv.writer(prediction_file)
prediction_file_object.writerow(['PassengerID', 'Survived'])
for row in test_file_object:
    if row[3] == 'female':
        prediction_file_object.writerow([row[0],'1']) #if female, predict survival
    else:
        prediction_file_object.writerow([row[0],'0']) #if male, predict they didn't survive

test_file.close()
prediction_file.close()
    

In [ ]:
fare_ceiling = 40
npdata[npdata[0::,9].astype(np.float) >= fare_ceiling, 9] = fare_ceiling - 1.0

fare_bracket_size = 10
number_of_price_brackets = fare_ceiling / fare_bracket_size

number_of_classes = len(np.unique(npdata[0::,2])) #count the number of unique classes

survival_table = np.zeros((2, number_of_classes, number_of_price_brackets))

In [ ]:
for i in xrange(number_of_classes):     
    for j in xrange(number_of_price_brackets):
        women_only_stats = npdata[ \
            (npdata[0::,4] == "female") & \
            (npdata[0::,2].astype(np.float) == i+1) & \
            (npdata[0:,9].astype(np.float) >= j*fare_bracket_size) & \
            (npdata[0:,9].astype(np.float) < (j+1)*fare_bracket_size),1]
        
        survival_table[0,i,j] = np.mean(women_only_stats.astype(np.float))

In [ ]:
for i in xrange(number_of_classes):
    for j in xrange(number_of_price_brackets):
        men_only_stats = npdata[ \
                               (npdata[0::,4] == 'male') & \
                               (npdata[0::,2].astype(np.float) == i+1) & \
                               (npdata[0:,9].astype(np.float) >= j*fare_bracket_size) & \
                               (npdata[0:,9].astype(np.float) < (j+1)*fare_bracket_size),1]
        
        survival_table[1,i,j] = np.mean(men_only_stats.astype(np.float))


In [ ]:
survival_table[ survival_table != survival_table ] = 0
survival_table[survival_table >= 0.5] = 1
survival_table[survival_table < 0.5] = 0
print survival_table

In [ ]:
test_file = open('test.csv', 'rb')
test_file_object = csv.reader(test_file)
header = test_file_object.next()

predictions_file = open("genderclassmodel.csv", "wb")
p = csv.writer(predictions_file)
p.writerow(["PassengerId", "Survived"])

for row in test_file_object:
    for j in xrange(number_of_price_brackets):
        try:
            row[8] = float(row[8])
        except:
            bin_fare = 3 - float(row[1])
            break
        if row[8] > fare_ceiling:
            bin_fare = number_of_price_brackets - 1
            break
        if row[8] >= j * fare_bracket_size and row[8] < (j+1) * fare_bracket_size:
            bin_fare = j
            break
            
        if row[3] == 'female':
            p.writerow([row[0], '%d' % int(survival_table[0, float(row[1]) - 1, bin_fare])])
        else:
            p.writerow([row[0], '%d' % int(survival_table[1, float(row[1]) - 1, bin_fare])])

test_file.close()
predictions_file.close()